In [1]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')

In [3]:
df.sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
100066,1793967731f73043,"]] Welcome to Wikipedia, Juhko! I have been e...",0,0,0,0,0,0
75060,c8d60eb5631f64a6,"""== Matt Cutts and the National Security Agenc...",0,0,0,0,0,0
144132,084f6ac9809da860,cHow so? Which one's specifically were?ome,0,0,0,0,0,0
155097,b8df76137b28a3bb,Exactly. And we'll also need to remove some of...,0,0,0,0,0,0
102294,2364680457450b67,Did you intend to remove all the content about...,0,0,0,0,0,0
134783,d0e7d4d28efbe3b5,"""\n\nI edited an extremely promotional section...",0,0,0,0,0,0
78197,d14e8115084bac11,"""\nUnlucky 13th; it just had to be that game f...",0,0,0,0,0,0
98725,101c4b2923402705,request the block be changed to a set time per...,0,0,0,0,0,0
28883,4c8b272cade83505,"""\n\n Improvement suggestion \n\nI suggest tha...",0,0,0,0,0,0
126821,a6350a3ca2c6c4ad,Quotes \n\nI removed the quotes section one o...,0,0,0,0,0,0


In [4]:
df.shape

(159571, 8)

In [5]:
df.iloc[0]['comment_text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

## Preprocess

In [6]:
from tensorflow.keras.layers import TextVectorization

In [7]:
# !pip list

In [8]:
# TextVectorization??

In [9]:
x=df['comment_text']

In [10]:
y=df[df.columns[2:]].values  

#values because at the end we want numpy array for model prediction

In [11]:
vectorizer=TextVectorization(max_tokens=200000,
                         output_sequence_length=1800, #maximum length of output sentence. chech the no. columns after vectorization
                         output_mode='int')

In [12]:
vectorizer.adapt(x.values)

In [13]:
# vectorizer('Mangesh is pursuing M.Sc. from bakwaas univeresity')

In [14]:
vectorize_text=vectorizer(x.values)

In [15]:
vectorize_text.shape

TensorShape([159571, 1800])

In [16]:
# Tensorflow pipeline

dataset=tf.data.Dataset.from_tensor_slices((vectorize_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [17]:
dataset.as_numpy_iterator().next()  #shows thee batch input. there are 16 rows as we gives 16 as abatch size

(array([[  410,     2,   248, ...,     0,     0,     0],
        [   94,    13,    10, ...,     0,     0,     0],
        [16457,   435,  1937, ...,     0,     0,     0],
        ...,
        [  312,     8,    65, ...,     0,     0,     0],
        [    7,   654,    20, ...,     0,     0,     0],
        [   50,     4,     2, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]]))

In [18]:
batch_x, batch_y=dataset.as_numpy_iterator().next()

In [19]:
batch_x.shape, batch_y.shape

((16, 1800), (16, 6))

In [20]:
len(dataset)  # 9974 batches are there

9974

In [21]:
train=dataset.take(int(len(dataset)*0.7))
val=dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test=dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

## Model building

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [23]:
model= Sequential()
model.add(Embedding(200000+1, 32))     #MAx_feature
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [24]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [26]:
history= model.fit(train, epochs=3, validation_data= val)

Epoch 1/3
6981/6981 [==============================] - 731s 103ms/step - loss: 0.0638 - val_loss: 0.0482
Epoch 2/3
6981/6981 [==============================] - 596s 85ms/step - loss: 0.0461 - val_loss: 0.0407
Epoch 3/3
6981/6981 [==============================] - 592s 85ms/step - loss: 0.0418 - val_loss: 0.0394


In [27]:
history.history

{'loss': [0.06381576508283615, 0.046116605401039124, 0.041769981384277344],
 'val_loss': [0.04820813611149788, 0.040714081376791, 0.0394221730530262]}

## Prediction

In [28]:
input_text=vectorizer("I will kill you, you don't know me")
model.predict(np.expand_dims(input_text,axis=0))

1/1 [==============================] - 1s 811ms/step


array([[0.874757  , 0.04118634, 0.3435632 , 0.06460708, 0.5615907 ,
        0.14237088]], dtype=float32)

In [36]:
batch=test.as_numpy_iterator().next()    # explanation

In [37]:
batch_x, batch_y =test.as_numpy_iterator().next()

In [38]:
(model.predict(batch_x) >0.5).astype('int')

1/1 [==============================] - 0s 76ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [40]:
batch_y    #for compairing

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

## Evaluate model

In [41]:
from tensorflow.keras.metrics import Precision, Recall ,CategoricalAccuracy

In [42]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    #unpack the batch
    x_true, y_true=batch
    # prediction
    yhat=model.predict(x_true)
    
    #flattern the prediction
    y_true=y_true.flatten()
    
    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat) 

In [44]:
print(f'precision {pre.result().numpy()}')

precision 0.8126826286315918


In [45]:
print(f'recall {re.result().numpy()}')

recall 0.7829391956329346


In [46]:
print(f'accuracy {acc.result().numpy()}')

accuracy 0.5195586681365967
